In [73]:
import pandas as pd
import numpy as numpy

import re
from bs4 import BeautifulSoup
from time import sleep
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

from selenium.common.exceptions import NoSuchElementException

import requests

import gender_guesser.detector as gender

import warnings
warnings.filterwarnings('ignore')


### Web scraping

#### Borial Management

In [230]:
url = "https://borialmanagement.com/"

In [231]:
res1 = requests.get(url)

In [232]:
res1.status_code

200

In [233]:
sopa = BeautifulSoup(res1.content, 'html.parser')

In [234]:
tags = sopa.find_all("h4", {"class": "personName toBold"})

In [235]:
actores = []

In [236]:
for tag in tags:
    actores.append(tag.text)

In [237]:
actores = list(set(actores))

In [238]:
actores2 = []

In [239]:
for actor in actores:
    if len(re.findall("\W", actor)) > 1:
        pass
    else:
        actores2.append(actor)

In [240]:
actores2.remove('GONZALO FERNГЃNDEZ')
actores2.remove('VГЌCTOR CASTILLO')
actores2.remove('ГЃLVARO QUINTANA')
actores2.remove('CARLA DГЌAZ')
actores2.remove('MARIO GARCГЌA')
actores2.remove('SANTI MARГЌN')
actores2.remove('ГЃLVARO FONTALBA')

In [241]:
actores3 = []

In [242]:
for actor in actores2:
    actores3.append(actor.split())

In [243]:
d = gender.Detector()

nombres_por_genero = {"masculino": [], "femenino": []}

for nombre in actores3:
    nombre_completo = nombre[0].capitalize() + nombre[1].capitalize()
    genero = d.get_gender(nombre[0].capitalize())
    if genero == "male":
        nombres_por_genero["masculino"].append(nombre_completo)
    elif genero == "female":
        nombres_por_genero["femenino"].append(nombre_completo)

print("Nombres de hombres:")
print(nombres_por_genero["masculino"])

print("Nombres de mujeres:")
print(nombres_por_genero["femenino"])

Nombres de hombres:
['JasonDay', 'PrinceEzeanyim', 'EnriqueAsenjo', 'CarlosGorbe', 'PatrickCriado', 'JorgeCalvo', 'JacoboCamarena', 'JorgeKent', 'DanielAlbaladejo', 'PedroCarvalho', 'JosepLinuesa']
Nombres de mujeres:
['AndreaDueso', 'MarinaVarem', 'NataliaHuarte', 'MarinaBellati', 'EvaIsanta', 'AnaBokesa', 'FedeCelada']


In [245]:
opciones= Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
#para ocultarme como robot
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_argument('--start-maximized') #empezar maximizado
opciones.add_argument('user.data-dir=selenium') #guarda las cookies
opciones.add_argument('--incognito')#incognito window

dicc = {}

df = pd.DataFrame(columns=["artista", "medio", "programa", "produccion/director"])
df_total = pd.DataFrame(columns=["artista", "medio", "programa", "produccion/director"])
dataframes = []

for k, v in nombres_por_genero.items():
    if k == "masculino":
        for k in range(len(v)):
            url = f"https://borialmanagement.com/actores/{nombres_por_genero['masculino'][k]}"
            driver = webdriver.Chrome(ChromeDriverManager().install())
            driver.get(url)
            driver.implicitly_wait(10)
            resultado = driver.find_element("css selector", "body > div:nth-child(3) > div > div:nth-child(2) > div").text
            resultado = resultado.split("\n")
            artista = resultado[0].lower()
            resultado.pop(0)

            for item in resultado:
                if item == 'CINE':
                    clave_actual = 'cine'
                    dicc[clave_actual] = []
                elif item == 'TELEVISIÓN':
                    clave_actual = 'television'
                    dicc[clave_actual] = []
                elif item == 'TEATRO':
                    clave_actual = 'teatro'
                    dicc[clave_actual] = []
                elif item == 'CORTOMETRAJES':
                    clave_actual = 'cortometrajes'
                    dicc[clave_actual] = []
                elif item == 'FORMACIÓN':
                    clave_actual = 'formacion'
                    dicc[clave_actual] = []
                elif item == 'VIDEOCLIP':
                    clave_actual = 'videoclip'
                    dicc[clave_actual] = []
                else:
                    dicc[clave_actual].append(item)

            for key in dicc.keys():
                for i in range(len(dicc[key])):
                    if type(dicc[key][i]) == list: 
                        dicc[key][i] = ".".join(dicc[key][i]).split(".", 1)
                    else:
                        dicc[key][i] = dicc[key][i].split(".", 1)


            for medio, programas in dicc.items():
                for programa in programas:
                    if len(programa) < 2:
                        pass
                    else:
                        df = df.append({"artista": artista, "medio": medio, "programa": programa[0], "produccion/director": programa[1]}, ignore_index=True)
                        dataframes.append(df)
    else:
        for k in range(len(v)):
            url = f"https://borialmanagement.com/actrices/{nombres_por_genero['femenino'][k]}"
            driver = webdriver.Chrome(ChromeDriverManager().install())
            driver.get(url)
            driver.implicitly_wait(10)
            resultado = driver.find_element("css selector", "body > div:nth-child(3) > div > div:nth-child(2) > div").text
            resultado = resultado.split("\n")
            artista = resultado[0].lower()
            resultado.pop(0)

            for item in resultado:
                if item == 'CINE':
                    clave_actual = 'cine'
                    dicc[clave_actual] = []
                elif item == 'TELEVISIÓN':
                    clave_actual = 'television'
                    dicc[clave_actual] = []
                elif item == 'TEATRO':
                    clave_actual = 'teatro'
                    dicc[clave_actual] = []
                elif item == 'CORTOMETRAJES':
                    clave_actual = 'cortometrajes'
                    dicc[clave_actual] = []
                elif item == 'FORMACIÓN':
                    clave_actual = 'formacion'
                    dicc[clave_actual] = []
                elif item == 'VIDEOCLIP':
                    clave_actual = 'videoclip'
                    dicc[clave_actual] = []
                else:
                    dicc[clave_actual].append(item)

            for key in dicc.keys():
                for i in range(len(dicc[key])):
                    if type(dicc[key][i]) == list: 
                        dicc[key][i] = ".".join(dicc[key][i]).split(".", 1)
                    else:
                        dicc[key][i] = dicc[key][i].split(".", 1)


            for medio, programas in dicc.items():
                for programa in programas:
                    if len(programa) < 2:
                        pass
                    else:
                        df = df.append({"artista": artista, "medio": medio, "programa": programa[0], "produccion/director": programa[1]}, ignore_index=True)
                        dataframes.append(df)
df = pd.concat(dataframes, axis=0)

#### Nara Talent

In [2]:
url = "http://www.elvirasanchezgallo.es/es/actores"
res1 = requests.get(url)
sopa = BeautifulSoup(res1.content, 'html.parser')
tags = sopa.find_all("div", {"class": "dj-slide-title"})

In [3]:
h_nara = []
for tag in tags:
    h_nara.append(tag.text)

In [4]:
h_nara2 = []
for actor in h_nara:
    actor = actor.strip("\r").strip("\n").strip("\t")
    h_nara2.append(actor)

In [5]:
url = "http://www.elvirasanchezgallo.es/es/actrices"
res1 = requests.get(url)
sopa = BeautifulSoup(res1.content, 'html.parser')
tags = sopa.find_all("div", {"class": "dj-slide-title"})

In [6]:
m_nara = []
for tag in tags:
    m_nara.append(tag.text)

In [7]:
m_nara2 = []
for actriz in m_nara:
    actriz = actriz.strip("\r").strip("\n").strip("\t")
    m_nara2.append(actriz)

In [8]:
nara = h_nara2 + m_nara2

In [9]:
m_nara2

['Itziar Atienza',
 'Carmen Barrantes',
 'Julia Carnero',
 'María Cerezuela ',
 'Getari Etxegarai',
 'Fiorella Faltoyano ',
 'Maggie García',
 'Laura Gómez-Lacueva',
 'María Jáimez',
 'Helena Kaittani',
 'Mercedes León ',
 'Petra Martinez',
 'Esmeralda Moya',
 'Blanca Portillo',
 'Ana Rayo',
 'Antea Rodríguez ',
 'Alicia Rodríguez',
 'Veronica Ronda ',
 'Ana Turpin',
 'Sara Valbuena']

In [10]:
nara2 = []
for act in nara:
    act = act.lower().strip().replace(" ", "-")
    nara2.append(act)

In [11]:
actores_nara = nara2

In [40]:
opciones= Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
#para ocultarme como robot
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_argument('--start-maximized') #empezar maximizado
opciones.add_argument('user.data-dir=selenium') #guarda las cookies
opciones.add_argument('--incognito')#incognito window

dataframes_nara = []
datos = {"formacion": [],
         "television": [],
         "cine": [],
         "cortometrajes": [],
         "teatro": []
         }


for i, actor in enumerate(actores_nara):
    artista = actor
    url = f"https://www.elvirasanchezgallo.es/es/{actor}"
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url)
    driver.implicitly_wait(10)
    driver.find_element("css selector", "#cookiehintsubmit").click()

    # teatro
    driver.find_element("css selector", "#sppb-ac-heading-1553890736287-key-0").click()
    datos1 = driver.find_elements("css selector", "#sppb-addon-1553793327882 > div > div > div > div > div > div > ul")
    for k in datos1:
        datos["teatro"].append(k.text)
    if datos["teatro"]:
        datos["teatro"][i] = "".join(datos["teatro"])
        datos["teatro"][i] = datos["teatro"].split("\n")
    else:
        datos["teatro"].append([])

    #cine
    driver.find_element("css selector", "#sppb-ac-heading-1553890736287-key-1").click()
    datos2 = driver.find_elements("css selector", "#sppb-addon-1553626155274 > div > div > ul")
    for kk in datos2:
        datos["cine"].append(kk.text)
    datos["cine"] = "".join(datos["cine"])
    datos["cine"] = datos["cine"].split("\n")

    # cortometrajes
    driver.find_element("css selector", "#sppb-ac-heading-1553890736287-key-3").click()
    datos3 = driver.find_elements("css selector", "#sppb-addon-1553628841525 > div > div > ul")
    for kkk in datos3:
        datos["cortometrajes"].append(kkk.text)
    datos["cortometrajes"] = "".join(datos["cortometrajes"])
    datos["cortometrajes"] = datos["cortometrajes"].split("\n")

    # televisión
    driver.find_element("css selector", "#sppb-ac-heading-1553890736287-key-2").click()
    datos4 = driver.find_elements("css selector", "#sppb-addon-1553626155273 > div > div > ul")
    for kkkk in datos4:
        datos["television"].append(kkk.text)
    datos["television"] = "".join(datos["television"])
    datos["television"] = datos["television"].split("\n")

    # formación
    driver.find_element("css selector", "#sppb-ac-heading-1553890736287-key-4").click()
    datos5 = driver.find_elements("css selector", "#sppb-addon-1553628896794 > div > div > ul")
    for kkkkk in datos5:
        datos["formacion"].append(kkk.text)
    datos["formacion"] = "".join(datos["formacion"])
    datos["formacion"] = datos["formacion"].split("\n")


    for key in datos.keys():
        for i in range(len(datos[key])):
            for j in range(len(datos[key][i])):
                datos[key][i][j] = datos[key][i][j].split('"', 2)

    for medio, programas in datos.items():
        for programa in programas:
            if len(programa) < 2:
                pass
            else:
                df = df.append({"artista": artista, "medio": medio, "programa": programa[0], "produccion/director": programa[1]}, ignore_index=True)
                dataframes.append(df)
df = pd.concat(dataframes, axis=0)

TypeError: sequence item 0: expected str instance, list found

In [41]:
opciones= Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
#para ocultarme como robot
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_argument('--start-maximized') #empezar maximizado
opciones.add_argument('user.data-dir=selenium') #guarda las cookies
opciones.add_argument('--incognito')#incognito window


dataframes_nara = []
datos = {"formacion": [],
         "television": [],
         "cine": [],
         "cortometrajes": [],
         "teatro": []
         }

url = "https://www.elvirasanchezgallo.es/es/ana-turpin"
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)
driver.implicitly_wait(10)
driver.find_element("css selector", "#cookiehintsubmit").click()

# teatro
driver.find_element("css selector", "#sppb-ac-heading-1553626155268-key-0").click()
sleep(2)
datos1 = driver.find_elements("css selector", "#sppb-addon-1553793327882 > div > div > div > div > div > div > ul")
for k in datos1:
    datos["teatro"].append(k.text)
datos["teatro"] = "".join(datos["teatro"])
datos["teatro"] = datos["teatro"].split("\n")

#cine
driver.find_element("css selector", "#sppb-ac-heading-1553626155268-key-1").click()
sleep(2)
datos2 = driver.find_elements("css selector", "#sppb-addon-1553626155274 > div > div > ul")
for kk in datos2:
    datos["cine"].append(kk.text)
datos["cine"] = "".join(datos["cine"])
datos["cine"] = datos["cine"].split("\n")

# cortometrajes
driver.find_element("css selector", "#sppb-ac-heading-1553626155268-key-3").click()
sleep(2)
datos3 = driver.find_elements("css selector", "#sppb-addon-1553628841525 > div > div > ul")
for kkk in datos3:
    datos["cortometrajes"].append(kkk.text)
datos["cortometrajes"] = "".join(datos["cortometrajes"])
datos["cortometrajes"] = datos["cortometrajes"].split("\n")

# televisión
driver.find_element("css selector", "#sppb-ac-heading-1553626155268-key-2").click()
sleep(2)
datos4 = driver.find_elements("css selector", "#sppb-addon-1553626155273 > div > div > ul")
for kkkk in datos4:
    datos["television"].append(kkk.text)
datos["television"] = "".join(datos["television"])
datos["television"] = datos["television"].split("\n")

# formación
driver.find_element("css selector", "#sppb-ac-heading-1553626155268-key-4").click()
sleep(2)
datos5 = driver.find_elements("css selector", "#sppb-ac-content-1553626155268-key-4 > div")
sleep(2)
for kkkkk in datos5:
    datos["formacion"].append(kkk.text)
datos["formacion"] = "".join(datos["formacion"])
datos["formacion"] = datos["formacion"].split("\n")

In [42]:
datos

{'formacion': [''],
 'television': [''],
 'cine': ['"Ama" Dir. Julia de Paz Solvas. Prod. Ladalia Films. 2020',
  '"De púrpura y escarlata" Dir. Juanra Fernández, 2017',
  '"UMA" Dir. Alain Maiki, 2016',
  '"Autoréplica" Dir. Daniel Cabrero, 2013',
  '"Pare Elisa" DIr. Juanra Fernández, 2013',
  '"Deseos" Dir. Pepe Pavón, 2011',
  '"Trio de ases, el secreto de la Atlántida" Dir. Joseba Vázquez, 2007',
  '"Hot milk" Dir. Ricardo Bofill, 2005',
  '"El florido péncil" Dir. Juan José Porto, 2002'],
 'cortometrajes': ['"La Maleza" de Carlos Laín. 2022',
  '“A shoe story". Dir: Eva Vázquez de Reoyo. 2021',
  '"El vacio" Dir. Carlos Mayol, 2019',
  '"Mariposas verdes" Dir. Marina Testino, 2014',
  '"Ofelia" Dir. Anita Sinkovic, 2011',
  '"Toma nota" Dir. Miguel González Molina, 2005',
  '"Doctos Alonso" Dir. Jordán Bastoni, 2004',
  '"El principio del fin de tus días" Dir. Jesús Olmedo, 2004',
  '"O\' Donell 21" Dir. Yoël Dahan, 2003',
  '"Promoción prohibida de su venta" Dir. Luis Arribas, 2

#### Única Representaciones

In [14]:
url = "https://www.unicarepresentaciones.com/Actores"
res1 = requests.get(url)
sopa = BeautifulSoup(res1.content, 'html.parser')
tags = sopa.find_all("div", {"class": "views-field views-field-title"})

In [20]:
h_unica = []
for tag in tags:
    h_unica.append(tag.text)

In [25]:
for i in range(len(h_unica)):
    h_unica[i] = h_unica[i].strip().replace(" ", "-").lower()

In [31]:
url = "https://www.unicarepresentaciones.com/Actrices"
res1 = requests.get(url)
sopa = BeautifulSoup(res1.content, 'html.parser')
tags = sopa.find_all("div", {"class": "views-field views-field-title"})

In [32]:
m_unica = []
for tag in tags:
    m_unica.append(tag.text)

In [33]:
for i in range(len(m_unica)):
    m_unica[i] = m_unica[i].strip().replace(" ", "-").lower()

In [35]:
act_unica = h_unica+m_unica

In [246]:
opciones= Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_argument('--start-maximized')
opciones.add_argument('user.data-dir=selenium')
opciones.add_argument('--incognito')

dicc = {}
dataframes = []
df = pd.DataFrame()
for actor in act_unica:
    try:
        url = f"https://www.unicarepresentaciones.com/actores-actrices/{actor}"
        driver = webdriver.Chrome(ChromeDriverManager().install())
        driver.get(url)
        driver.implicitly_wait(10)
        cv = driver.find_element("css selector", "#block-system-main > div > div.ficha-actores > div.ficha-col-1").text

        elementos = cv.split("\n")
        elementos.pop(0)

        for item in elementos:
            if 'Cine' in item:
                clave = 'cine'
                dicc[clave] = []
            elif 'Televisión' in item:
                clave = 'television'
                dicc[clave] = []
            elif 'Teatro' in item:
                clave = 'teatro'
                dicc[clave] = []
            elif 'Cortometrajes' in item:
                clave = 'cortometrajes'
                dicc[clave] = []
            elif 'Formación' in item:
                clave = 'formacion'
                dicc[clave] = []
            elif 'Videoclip' in item:
                clave = 'videoclip'
                dicc[clave] = []
            else:
                dicc[clave].append(item)

        for key in dicc.keys():
            for i in range(len(dicc[key])):
                if type(dicc[key][i]) == list or "-" not in dicc[key][i]:
                    pass
                else:
                    dicc[key][i] = dicc[key][i].split("-", 1)

        artista = actor

        dataframes = []
        for medio, programas in dicc.items():
            for programa in programas:
                if len(programa) < 2:
                    pass
                else:
                    df = df.append({"artista": artista, "medio": medio, "programa": programa[0], "produccion/director": programa[1]}, ignore_index=True)
        dataframes.append(df)
    except NoSuchElementException:
        driver.close()

In [250]:
dataframes[0].artista.unique()

52

In [251]:
df_unica = dataframes[0]

In [252]:
df_unica.to_csv("../data/unica_raw.csv")

In [253]:
df_unica = pd.read_csv("../data/unica_raw.csv", index_col = 0)

In [256]:
df_unica.head()

,artista,medio,programa,produccion/director
0,javier-lorenzo,television,“CUÉNTAME CÓMO PASÓ”,(Oriol). Tve1. 2011-2021
1,javier-lorenzo,cine,“EL AVISO”,Dir. Daniel Calparsoro. 2018
2,javier-lorenzo,cine,”LIFE VEST UNDER YOUR SEAT” (Volamos hace Miami),Cortometraje - Dir. Maria Giraldez y Miguel P...
3,carlos-scholz,television,"""SIN LIMITES""",Amazon Prime Videos.
4,carlos-scholz,television,"""FERIA: LA LUZ MÁS OSCURA""",Filmax. Netflix


In [262]:
df_unica[df_unica.artista == "irene-rojo"]

,artista,medio,programa,produccion/director
976,irene-rojo,television,« YRREAL »,Dir. Alberto Utrera (Actriz de reparto: NATALIA)
977,irene-rojo,television,« ESTOY VIVO »,3ª Temporada (Actriz reparto - REBE). TVE
978,irene-rojo,television,« EL CONTINENTAL » Dir. Frank Ariza,(Actriz de reparto - MARÍA).TVE
979,irene-rojo,television,« INSTAMOM »,Webserie (actriz de reparto: INSTAGRAMER). P...
980,irene-rojo,television,« CENTRO MÉDICO»,TVE.
981,irene-rojo,television,« LQSA »,Telecinco.
982,irene-rojo,television,« SERVIR Y PROTEGER »,TVE.
983,irene-rojo,television,« LIKES »,Bailarina programa especial navidad / Movistar +
984,irene-rojo,cine,,
985,irene-rojo,cine,2,0


In [207]:
opciones= Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
#para ocultarme como robot
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_argument('--start-maximized') #empezar maximizado
opciones.add_argument('user.data-dir=selenium') #guarda las cookies
opciones.add_argument('--incognito')#incognito window


dataframes_nara = []
datos = {"formacion": [],
         "television": [],
         "cine": [],
         "cortometrajes": [],
         "teatro": []
         }

url = "https://www.unicarepresentaciones.com/actores-actrices/carlos-scholz"
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)
driver.implicitly_wait(10)
cv = driver.find_element("css selector", "#block-system-main > div > div.ficha-actores > div.ficha-col-1").text



In [208]:
elementos = cv.split("\n")
elementos.pop(0)

'Curriculum Vitae'

In [209]:
dicc = {}

for item in elementos:
    if 'Cine' in item:
        clave = 'cine'
        dicc[clave] = []
    elif 'Televisión' in item:
        clave = 'television'
        dicc[clave] = []
    elif 'Teatro' in item:
        clave = 'teatro'
        dicc[clave] = []
    elif 'Cortometrajes' in item:
        clave = 'cortometrajes'
        dicc[clave] = []
    elif 'Formación' in item:
        clave = 'formacion'
        dicc[clave] = []
    elif 'Videoclip' in item:
        clave = 'videoclip'
        dicc[clave] = []
    else:
        dicc[clave].append(item)

In [210]:
dicc

{'television': ['"SIN LIMITES" - Amazon Prime Videos.',
  '"FERIA: LA LUZ MÁS OSCURA" - Filmax. Netflix',
  '"DESCONOCIDAS" - (Pablo). Dir. Peris Romano. Canal Sur',
  '"JODIDÍSIMAS" - María Dueñas. Audible - Amazon.',
  '"TOY BOY"- Antena3. Netflix.'],
 'cine': ['“AWARENESS” - Daniel Benmayor - AmazonPrime - (protagonista) - 2023.',
  '”MI OTRO JON” - Pago Arango - 2023.',
  '"LA MANIOBRA DE LA TORTUGA" - Juan Miguel de Castillo - 2022.'],
 'teatro': ['"ROMEO Y JULIETA"- Dir. Patateatro',
  '"COPLAS DE PABLO RUÍZ PICASSO"- Dir: Ángel Baena',
  '"CHICAGO, EL MUSICAL"- Dir: Celia Dolci',
  '"LA CASA HABITADA"- Dir. Ángel Baena',
  ' ']}

In [211]:
for key in dicc.keys():
    for i in range(len(dicc[key])):
        dicc[key][i] = dicc[key][i].split("-", 1)

In [213]:
df = pd.DataFrame()

In [214]:
artista = "carlos-scholz"

In [215]:
dataframes = []
for medio, programas in dicc.items():
    for programa in programas:
        if len(programa) < 2:
            pass
        else:
            df = df.append({"artista": artista, "medio": medio, "programa": programa[0], "produccion/director": programa[1]}, ignore_index=True)
dataframes.append(df)

In [217]:
dataframes[0]

,artista,medio,programa,produccion/director
0,carlos-scholz,television,"""SIN LIMITES""",Amazon Prime Videos.
1,carlos-scholz,television,"""FERIA: LA LUZ MÁS OSCURA""",Filmax. Netflix
2,carlos-scholz,television,"""DESCONOCIDAS""",(Pablo). Dir. Peris Romano. Canal Sur
3,carlos-scholz,television,"""JODIDÍSIMAS""",María Dueñas. Audible - Amazon.
4,carlos-scholz,television,"""TOY BOY""",Antena3. Netflix.
5,carlos-scholz,cine,“AWARENESS”,Daniel Benmayor - AmazonPrime - (protagonista...
6,carlos-scholz,cine,”MI OTRO JON”,Pago Arango - 2023.
7,carlos-scholz,cine,"""LA MANIOBRA DE LA TORTUGA""",Juan Miguel de Castillo - 2022.
8,carlos-scholz,teatro,"""ROMEO Y JULIETA""",Dir. Patateatro
9,carlos-scholz,teatro,"""COPLAS DE PABLO RUÍZ PICASSO""",Dir: Ángel Baena
